Mengunduh Library

In [ ]:
! pip install sentence-splitter
! pip install transformers
! pip install SentencePiece
! pip install torch
! pip install deep-translator
! pip install datasets transformers[sentencepiece]
! pip install sacrebleu
! pip install -U sentence-transformers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 54.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 30.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 33.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 11.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 82.9 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling nvidia-nvjitlink-cu12-12.5.82:
      Successfully uninstalled nvidia-nvjitlin

Menginport Library dan Memulai Data Model Parafrase dan Fungsi Parafrase

In [ ]:
import torch
from transformers import PegasusForConditionalGeneration, PegasusTokenizer

model_name = 'tuner007/pegasus_paraphrase'
torch_device = 'cuda' if torch.cuda.is_available() else 'cpu'
tokenizer = PegasusTokenizer.from_pretrained(model_name)
model = PegasusForConditionalGeneration.from_pretrained(model_name).to(torch_device)

def get_response(input_text,num_return_sequences):
    batch = tokenizer.prepare_seq2seq_batch([input_text],truncation=True,padding='longest',max_length=60, return_tensors='pt').to(torch_device)
    translated = model.generate(**batch,max_length=60,num_beams=10, num_return_sequences=num_return_sequences, temperature=1.6)
    tgt_text = tokenizer.batch_decode(translated, skip_special_tokens=True)
    return tgt_text

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/86.0 [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/1.91M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/65.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.14k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/2.28G [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.28G [00:00<?, ?B/s]

Some weights of PegasusForConditionalGeneration were not initialized from the model checkpoint at tuner007/pegasus_paraphrase and are newly initialized: ['model.decoder.embed_positions.weight', 'model.encoder.embed_positions.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Memulai fungsi penerjemah untuk memparafrasekan model

In [ ]:
from deep_translator import GoogleTranslator

def solve(param, r_seq, src='id', dest='en') :
    print('Hasil Parafrase\n---------------\n- Sequence : {}\n- Raw Text : {}\n---------------'.format(r_seq, param))
    src_translator = GoogleTranslator(source=src, target=dest).translate
    dest_translator = GoogleTranslator(source=dest, target=src).translate
    param = src_translator(param)
    result = get_response(param, r_seq)
    for idx, i in enumerate(result) :
        idx += 1
        i = dest_translator(i)
        print('{}. {}'.format(idx, i))

Hasil Parafrase

In [ ]:
text = 'saya suka makan naasi.	 '
sequence = 10
solve(text, sequence)

Hasil Parafrase
---------------
- Sequence : 10
- Raw Text : saya suka makan naasi.	 
---------------
1. Saya menikmati makan naasi.
2. Saya suka makan makanan.
3. Saya suka makan.
4. Saya suka makan camilan.
5. Saya suka makan naasi.
6. Saya suka makan mie.
7. Saya suka makan naasi.
8. Saya suka sarapan.
9. Saya suka makan.
10. Saya suka makan nasi.


Menghitung Bleu Score

In [ ]:
from datasets import load_metric

ImportError: cannot import name 'load_metric' from 'datasets' (/usr/local/lib/python3.11/dist-packages/datasets/__init__.py)

In [ ]:
sacrebleu = load_metric("sacrebleu")
# SacreBLEU operates on raw text, not tokens
predictions = ["i like playing football "]
references = [["i like  playing football "]]
sacrebleu.compute(predictions=predictions, references=references)

{'score': 100.00000000000004,
 'counts': [4, 3, 2, 1],
 'totals': [4, 3, 2, 1],
 'precisions': [100.0, 100.0, 100.0, 100.0],
 'bp': 1.0,
 'sys_len': 4,
 'ref_len': 4}

Menghitung Semantic Similarity

In [ ]:
from sentence_transformers import SentenceTransformer, util
sentences = ["i like playing football", "i hate playing football"]

model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')

In [ ]:
embedding_1= model.encode(sentences[0], convert_to_tensor=True)
embedding_2 = model.encode(sentences[1], convert_to_tensor=True)

util.pytorch_cos_sim(embedding_1, embedding_2)

tensor([[0.7926]])

Read CSV

In [ ]:
import pandas as pd

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
df = pd.read_csv('/content/gdrive/MyDrive/Data_Testing/50_Kalimat.csv')
df

,No,Inggris,parafrase,Referensi,Bleu Score,Semantic Similarity
0,1,"A westward drive along Wensleydale, taken in v...",Keindahan alami lembah ini ditampilkan melalui...,"Perjalanan ke barat di sepanjang Wensleydale, ...",55.35,0.84
1,2,"The hills, dotted with sheep munching on the c...","Perbukitan, dihiasi dengan domba yang mengunya...","Bukit-bukit, dihiasi dengan domba yang menguny...",61.47,0.78
2,3,"Once pitched, we can survey our surroundings a...",Kita dapat menikmati apa yang ditawarkan kota ...,"Setelah melenggang, kita dapat mengamati sekel...",55.27,"0,75"
3,4,It must have been a fantastic journey when the...,"Ada pameran statis mesin uap dan dua gerbong, ...",Itu pasti perjalanan yang fantastis ketika jal...,45.27,0.87
4,5,"Why would someone, anyone, build a town in suc...",Mengapa ada orang yang membangun kota di daera...,"Mengapa seseorang, siapa pun, membangun kota d...",75.35,0.72
5,6,It must be wild up here in winter?” These stat...,"""Pasti liar di sini di musim dingin?"" adalah ...","Pasti liar di sini di musim dingin?"" Pernyataa...",52.61,0.82
6,7,The little market town became a hub and thus i...,Kota pasar kecil menjadi pusatnya dan akan sel...,Kota pasar kecil menjadi pusat dan dengan demi...,85.43,0.73
7,8,Resulting prosperity from bygone days is seen ...,"Banyak bangunan didedikasikan untuk seni, kera...",Kemakmuran yang dihasilkan dari masa lalu terl...,75.64,0.65
8,9,"Sadly, the wonderful Ropemaker has not yet re-...",The Ropemaker belum dibuka kembali untuk umum ...,"Sayangnya, Ropemaker yang luar biasa belum dib...",50.42,0.83
9,10,"Most impressive of these, is an artisan who ma...",Seorang pengrajin yang membuat berbagai instru...,"Yang paling mengesankan dari ini, adalah seora...",58.93,0.93


In [ ]:
df = pd.read_csv('/content/gdrive/MyDrive/Data_Testing/testdata_100.csv')
df

,NO,INGGRIS,PARAFRASE,REFERENSI,BLEU SCORE,SEMANTIC SIMILARITY
0,1,You can do it,kamu mampu melakukannya,kamu bisa melakukannya,75.35,0.89
1,2,you have what it takes,anda tahu apa yang di perlukan,anda memiliki apa yang di perlukan,53.72,0.92
2,3,you are capable of greatness,anda bisa menjadi hebat,anda mampu menjadi hebat,75.27,0.88
3,4,you are worth fighting for,anda layak mendapatkannya,anda layak diperjuangkan,55.35,0.77
4,5,You are capable of amazing things,anda mampu melakukan hal hal luar biasa,anda mampu melakukan hal hal menakjubkan,61.47,0.84
...,...,...,...,...,...,...
95,96,I’m doing my homework,aku sedang belajar,aku sedang mengerjakan PR,55.27,0.67
96,97,I woke up late this morning.,aku bangun siang hari ini,aku bangun terlambat tadi pagi,25.36,0.74
97,98,I drank a glass of milk an hour ago.,satu jam yang lalu saya minum susu,aku minum segelas susu sejam yang lalu,20.61,0.70
98,99,I will buy some food.,saya akan membayar makanan,aku akan beli makanan,25.99,0.77


In [ ]:
df = pd.read_csv('/content/gdrive/MyDrive/Data_Testing/100_Kalimat.csv')
df

,No,Inggris,parafrase
0,1,"Hello, how are you?","hai, apa kabar"
1,2,Nice to meet you,bahagia bisa bersua denganmu
2,3,What is your name?,siapa anda?
3,4,Where do you live?,dimana kamu bersemayam?
4,5,What is your father?,apa proyek ayahmu?
...,...,...,...
197,96,Whats your hobby?,apakah kegemaran kamu?
198,97,Do you like any sport?,apa kamu suka basket?
199,98,Do you understand my point?,apa kamu tau rencana saya?
200,99,Talk to you later.,kita berbincang lagi di hari lain
